In [1]:
%pylab inline

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import sqlite3
import pandas as pd

import surprise

import utilities as util

Populating the interactive namespace from numpy and matplotlib


# Load data

In [2]:
filename = '../data/mountain_project_gunks.sqlite'
conn = sqlite3.connect(filename)

# Climber profiles
df_users = pd.read_sql('SELECT * FROM users', conn)

# Route data
df_items = pd.read_sql("""
SELECT items.*, pro.pro_type
FROM items JOIN pro 
ON items.pro_id = pro.id
""", conn)

# Ratings of each route
df_ratings = pd.read_sql('SELECT * FROM ratings', conn)

In [3]:
display(df_users.head(3).style.format(util.make_clickable, subset=['user_url']))
display(df_items.drop('description', axis=1).head(3).style.format(util.make_clickable, subset=['route_url']))
display(df_ratings.head(3))

,uid,name,user_url,profile
0,0,Mark Roth,https://www.mountainproject.com/user/106087077/mark-roth,Boulder Male
1,1,Joel Ryan,https://www.mountainproject.com/user/109581184/joel-ryan,"New York, New York 24 years old Male"
2,2,Anyo Lesiuk,https://www.mountainproject.com/user/200223567/anyo-lesiuk,"Philadelphia, PA"


,iid,route_name,route_url,difficulty,pro_id,length,pitches,nratings,monthly_views,total_views,pro_type
0,0,Sudoriferous,https://www.mountainproject.com/route/108300801/sudoriferous,5.2,1,120,2,7,19,1309,Trad
1,1,Cordelette Arete,https://www.mountainproject.com/route/112109817/cordelette-arete,5.2,1,80,1,12,38,1161,Trad
2,2,69,https://www.mountainproject.com/route/106133868/69,5.3,1,50,1,59,36,4727,Trad


,uid,iid,rating
0,0,0,2
1,1,0,2
2,2,0,2


# Load user-based and item-based models

In [4]:
filename ='../data/model_knn_user_based'
_, algo_user = surprise.dump.load(filename)

filename ='../data/model_knn_item_based'
_, algo_item = surprise.dump.load(filename)

In [5]:
uid = 0
iid = 0

print(algo_user.predict(uid, iid).est)
print(algo_user.sim_options)
print(algo_user.sim.shape)

print(algo_item.predict(uid, iid).est)
print(algo_item.sim_options)
print(algo_item.sim.shape)

1.6388636861275163
{'name': 'msd', 'user_based': True}
(2432, 2432)
2.028957400871496
{'name': 'msd', 'user_based': False}
(470, 470)


# Top n route recommendations
Predict the routes that a climber will like the most.

In [8]:
uid = 1237
n = 5

# Display climber info
df = df_users[df_users['uid']==uid][['name', 'user_url']]
display(df.style.format(util.make_clickable, subset=['user_url']))

# Display route info
df = util.recommend_top_n_routes(uid, algo_user, df_items, n=n)
display(df.style.format(util.make_clickable, subset=['route_url']))

,name,user_url
1237,megan chesher,https://www.mountainproject.com/user/110901677/megan-chesher


,iid,rating_est,route_name,route_url,difficulty,pro_id,length,pitches,nratings,monthly_views,total_views,pro_type
0,431,4.10002,Bone Hard,https://www.mountainproject.com/route/111476310/bone-hard,5.12b,1,80,1,7,35,1349,Trad
1,301,4.06578,Vanishing Point,https://www.mountainproject.com/route/115446450/vanishing-point,5.12a,1,100,1,3,46,213,Trad
2,299,4.02046,Crack' N Up,https://www.mountainproject.com/route/112426662/crack-n-up,5.11d,1,90,1,3,33,880,Trad
3,393,3.95824,Kligfield's Follies,https://www.mountainproject.com/route/107486305/kligfields-follies,5.11d,1,75,1,13,48,4100,Trad
4,363,3.94024,The Yellow Wall,https://www.mountainproject.com/route/105798924/the-yellow-wall,5.11c,1,125,2,37,136,21656,Trad


# Suggest climbing partners
Find the climbing partners that are most like a given climber. This can be added to the Mountain Project [partner finder](https://www.mountainproject.com/partner-finder).

In [10]:
# Generate a list of climbers looking for partners
df_looking_for_partners = df_users.sample(20, random_state=0)
df_looking_for_partners.head(3)

,uid,name,user_url,profile
672,672,Kimberly Blakemore,https://www.mountainproject.com/user/112285206...,None
2293,2293,BrianUp Up,https://www.mountainproject.com/user/106000626...,"Charlottesville, VA Male"
841,841,Kristike K,https://www.mountainproject.com/user/107277941...,Nyc 33 years old Female


In [11]:
# Find other climbers most similar to the climber.
uid = 1237

# Climber info
display(df_users[df_users['uid']==uid].style.format(util.make_clickable, subset=['user_url']))

# Potential partner info
df_ranked_partners = util.rank_partners_by_similarity(uid, algo_user, df_looking_for_partners)
display(df_ranked_partners.style.format(util.make_clickable, subset=['user_url']))

,uid,name,user_url,profile
1237,1237,megan chesher,https://www.mountainproject.com/user/110901677/megan-chesher,"frederick, md 28 years old Female"


,uid,name,user_url,profile,inner_uid,similarity
10,2201,Jon Nic,https://www.mountainproject.com/user/112001335/jon-nic,35 years old Male,2201,1
14,2207,Jordan Kessler,https://www.mountainproject.com/user/108104088/jordan-kessler,"Westerville, OH 32 years old Male",2207,1
9,182,Bob Graham,https://www.mountainproject.com/user/105873502/bob-graham,"Portland, OR",182,1
6,991,Jenna NY,https://www.mountainproject.com/user/108020482/jenna-ny,36 years old Female,991,1
18,674,Elliott Becker,https://www.mountainproject.com/user/110839245/elliott-becker,Washington 36 years old Male,674,0.857143
8,518,Mike Caruso,https://www.mountainproject.com/user/106117267/mike-caruso,Male,518,0.727273
3,758,Harry Netzer,https://www.mountainproject.com/user/107412430/harry-netzer,"Roxbury, CT 26 years old Male",758,0.714286
19,735,Larry S,https://www.mountainproject.com/user/106778424/larry-s,"Easton, PA 35 years old Male",735,0.571429
1,2293,BrianUp Up,https://www.mountainproject.com/user/106000626/brianup-up,"Charlottesville, VA Male",2293,0.5
4,148,Medic741,https://www.mountainproject.com/user/107536174/medic741,"Des Moines, IA (WTF) 26 years old Male",148,0.5


# Find routes most similar to a given route

In [29]:
iid = 30

# Climber info
display(df_items[df_items['iid']==iid].drop('description', axis=1).style.format(util.make_clickable, subset=['route_url']))

# Potential partner info
df_ranked_routes = rank_routes_by_similarity(iid, algo_item, df_items)
display(df_ranked_routes.drop('description', axis=1).head(60).style.format(util.make_clickable, subset=['route_url']))

,iid,route_name,route_url,difficulty,pro_id,length,pitches,nratings,monthly_views,total_views,pro_type
30,30,Nice 5.9 Climb,https://www.mountainproject.com/route/106512607/nice-59-climb,5.9,1,50,1,43,19,2163,Trad


,iid,route_name,route_url,difficulty,pro_id,length,pitches,nratings,monthly_views,total_views,pro_type,inner_iid,similarity
373,373,Sticky Gate,https://www.mountainproject.com/route/111281525/sticky-gate,5.5,1,160,2,2,7,293,Trad,373,1
30,30,Nice 5.9 Climb,https://www.mountainproject.com/route/106512607/nice-59-climb,5.9,1,50,1,43,19,2163,Trad,30,1
98,98,Sultana,https://www.mountainproject.com/route/108320281/sultana,5.8,1,150,2,6,5,319,Trad,98,1
462,462,Unknown aka Ken and the Art of Motorcycle Maintenance,https://www.mountainproject.com/route/109672957/unknown-aka-ken-and-the-art-of-motorcycle-maintenance,5.10b,2,50,1,3,13,679,TR,462,1
274,274,Android-Moby Dick link-up,https://www.mountainproject.com/route/106711001/android-moby-dick-link-up,5.8,1,0,2,6,11,1185,Trad,274,1
411,411,Emilio,https://www.mountainproject.com/route/111787942/emilio,5.7+,1,250,3,4,13,441,Trad,411,1
460,460,Squiggles Direct,https://www.mountainproject.com/route/106138061/squiggles-direct,5.10b,1,50,1,20,13,1686,Trad,460,1
194,194,Bitchy Virgin,https://www.mountainproject.com/route/105830332/bitchy-virgin,5.5,1,160,2,28,18,2844,Trad,194,1
95,95,Uncle Rudy,https://www.mountainproject.com/route/108337993/uncle-rudy,5.7+,1,210,3,14,15,989,Trad,95,1
459,459,DD Route,https://www.mountainproject.com/route/107524902/dd-route,5.10b,1,50,1,9,11,911,Trad,459,1
